In [1]:
import pandas as pd
df = pd.read_csv("~/desktop/apartment_crawler/top_20_data.csv")

In [ ]:
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import geopy.geocoders
geopy.geocoders.options.default_timeout = 10

geolocator = Nominatim(user_agent="petfees")
df['address_cleaned'] = df['address'].str.split('–').apply(lambda x: x[0])
df['geopy'] = df['address_cleaned'].progress_apply(lambda address: geolocator.geocode(address, addressdetails=True)) # Can take ~50min
df['longitude'] = df['geopy'].parallel_apply(lambda geopy_address: geopy_address.longitude if geopy_address else None)
df['latitude'] = df['geopy'].parallel_apply(lambda geopy_address: geopy_address.latitude if geopy_address else None)
df['spatialjusticetemplate'] = df.parallel_apply(lambda row: template.format(row['latitude'], row['longitude']), axis=1)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


  0%|          | 0/5911 [00:00<?, ?it/s]

In [ ]:
import pickle as pkl
with open("geo.pkl", "wb") as fp:
    pkl.dump(df, fp)

In [ ]:
import requests
import json
df['census'] = df.progress_apply(lambda row: json.loads(requests.get(row['spatialjusticetemplate']).content.decode("utf-8")), axis=1)